## Load Module Speaker Recognition


In [5]:
import neural_net
import inference
from collections import defaultdict, Counter

In [6]:
encoder_path = r"D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\Speaker_Recognition\LSTM\saved_model\train-clean-360-hours-50000-epochs-specaug-8-batch-3-stacks-cpu\mfcc_lstm_model_360h_50000epochs_specaug_8batch_3stacks_cpu.pt"
speaker_recognition_model = neural_net.get_speaker_encoder(encoder_path)


In [7]:
tri_base_embedding = inference.get_embedding(r"D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\Speaker_Recognition\LSTM\Data Tiếng nói tổng hợp\Trí\tri_1.wav", speaker_recognition_model)
dat_base_embedding = inference.get_embedding(r"D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\Speaker_Recognition\LSTM\Data Tiếng nói tổng hợp\Đạt\DAT_7.wav", speaker_recognition_model)
tuan_base_embedding = inference.get_embedding(r"D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\Speaker_Recognition\LSTM\Data Tiếng nói tổng hợp\Tuấn\tuan_7.wav", speaker_recognition_model)
phat_base_embedding = inference.get_embedding(r"D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\Speaker_Recognition\LSTM\Data Tiếng nói tổng hợp\Phát\phat_1.wav", speaker_recognition_model)

## Load Module Speech Recognition

In [8]:
import torch
from transformers import pipeline

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
speech_recognition_model = pipeline('automatic-speech-recognition', model='vinai/PhoWhisper-base', device=DEVICE)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
def speech2text(wav_file, model):
    return model(wav_file)['text']

## Load Module PoS Tagging


In [11]:
import pickle
from pyvi import ViTokenizer
import unicodedata as ud
import re

In [12]:
with open('D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\PoS_Tagging\Model\wordBank_vi.pkl', 'rb') as f:
    wordBank = pickle.load(f)

with open('D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\PoS_Tagging\Model\mostCommonBigrams_vi.pkl', 'rb') as f:
    mostCommonBigrams = pickle.load(f)


In [13]:
def Viterbi_rule_based(word,wordIdx, lineSize, line):
    """
    Hàm này tạo vector đặc trưng cho từng từ dựa trên quy tắc Viterbi.

    Parameters:
        word (str): Từ hiện tại.
        wordIdx (int): Vị trí của từ trong câu.
        lineSize (int): Độ dài của câu.
        line (list): Danh sách các từ trong câu.

    Returns:
        list: Vector đặc trưng cho từ.
    """
    feat = [1]
    sentPercent = float(wordIdx)/float(lineSize)
    feat.append(sentPercent)

    if word[0].isupper() and wordIdx != 0:
        feat.append(1)
    else:
        feat.append(0)

    posIdx_array = ([0] * len(tagsetDict))
    posSet = []
    if word in wordBank:
        posSet = wordBank[word]
    else:
        if wordIdx == 0:
            posSet = list(tagsetDict.keys())[0]
            posIdx_array[tagsetDict[posSet]] = 1
            return feat + posIdx_array + [0]
        else:
            prevWord = line[wordIdx-1]
            if prevWord in wordBank:
                prevPos = wordBank[prevWord]
                maxPos = mostCommonBigrams[prevPos]
                if prevPos == "E":
                    maxPos = 3
                posIdx_array[tagsetDict[maxPos]] = 1
                feat += posIdx_array + [tagsetDict[maxPos]]
                return feat
            else:
                posSet = list(tagsetDict.keys())[0]
                posIdx_array[tagsetDict[posSet]] = 1
                feat += posIdx_array + [0]
                return feat

    for pos in posSet:
        posIdx = tagsetDict[pos]
        posIdx_array[posIdx] += 1.0/len(wordBank[word])
    feat += (posIdx_array) + [0]
    return feat

def syllablize(sentence):
  """
    Hàm này chia câu thành các âm tiết.

    Parameters:
        sentence (str): Câu cần chia.

    Returns:
        list: Danh sách các âm tiết trong câu.
    """
  word = '\w+'
  non_word = '[^\w\s]'
  digits = '\d+([\.,_]\d+)+'

  patterns = []
  patterns.extend([word, non_word, digits])
  patterns = f"({'|'.join(patterns)})"

  sentence = ud.normalize('NFC', sentence)
  tokens = re.findall(patterns, sentence, re.UNICODE)
  return [token[0] for token in tokens]

def longest_matching(sentence, bi_grams, tri_grams):
  """
    Hàm này tìm các từ ghép dài nhất trong câu.

    Parameters:
        sentence (str): Câu cần kiểm tra.
        bi_grams (list): Danh sách các bi-grams.
        tri_grams (list): Danh sách các tri-grams.

    Returns:
        list: Danh sách các từ ghép dài nhất trong câu.
    """
  syllables = syllablize(sentence)
  syl_len = len(syllables)

  curr_id = 0
  word_list = []
  done = False

  while (curr_id < syl_len) and (not done):
    curr_word = syllables[curr_id]
    if curr_id >= syl_len - 1:
      word_list.append(curr_word)
      done = True
    else:
      next_word = syllables[curr_id + 1]
      pair_word = " ".join([curr_word.lower(), next_word.lower()])
      if curr_id >=(syl_len - 2):
        if pair_word in bi_grams:
          word_list.append("_".join([curr_word, next_word]))
          curr_id += 2
        else:
          word_list.append(curr_word)
          curr_id += 1
      else:
        next_next_word = syllables[curr_id + 2]
        triple_word = ' '.join([pair_word, next_next_word.lower()])
        if triple_word in tri_grams:
          word_list.append('_'.join([curr_word, next_word, next_next_word]))
          curr_id += 3
        elif pair_word in bi_grams:
          word_list.append('_'.join([curr_word, next_word]))
          curr_id += 2
        else:
          word_list.append(curr_word)
          curr_id += 1
  return word_list

def toString(wl, model, bi_grams, tri_grams,tagsetDict, inverseTagsetDict):
  """
    Hàm này chuyển đổi danh sách các từ thành chuỗi.

    Parameters:
        wl (list): Danh sách các từ cần chuyển đổi.

    Returns:
        str: Chuỗi kết quả.
    """
  wl = longest_matching(wl, bi_grams, tri_grams)
  X = []
  A = []
  text = ''
  for i in set(wl):
    if i == '.':
      A = './.'
    else:
      X.append(Viterbi_rule_based(i, 1, 1, 1))
      A = str(model.predict(X))
  for i in range(len(wl)):
    text += wl[i]
    text +='/'
    text +=str(A[2])
    text += ' '

  return wl[i], str(A[2])

def load_from_txt(filename):
    arr = []
    with open(filename, 'r') as f:
        for line in f:
            arr.append(line.strip())
    return arr

def load_ngrams_from_files():
    bi_grams = load_from_txt(r'D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\PoS_Tagging\Dataset\bi_grams_vi.txt')
    tri_grams = load_from_txt(r'D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\PoS_Tagging\Dataset\tri_grams_vi.txt')
    qua_grams = load_from_txt(r'D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\PoS_Tagging\Dataset\qua_grams_vi.txt')
    return bi_grams, tri_grams, qua_grams

def load_dict_from_txt(filename):
    dictionary = {}
    with open(filename, 'r') as f:
        for line in f:
            key, value = line.strip().split(':')
            dictionary[key] = int(value)
    return dictionary



In [14]:
with open(r'D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\PoS_Tagging\Model\postagging_SVM_Vi.pkl', 'rb') as f:
  postagging_model = pickle.load(f)

# Load từ điển tagset từ file văn bản
tagsetDict = load_dict_from_txt(r'D:\Code\BachKhoa\PBL 5\PBL05_smart_home_with_voice_print_and_antifraud_ai\AI Module\PoS_Tagging\Dataset\tagsetDict_vi.txt')

# Tạo từ điển nghịch đảo cho tagsetDict
inverse_tagsetDict = {tagsetDict[k]: k for k in tagsetDict}



c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator On

In [15]:
def sentence_to_pos(sentence, model):
    # Load các n-grams từ các tệp
    bi_grams, tri_grams, qua_grams = load_ngrams_from_files()
    
    # Khởi tạo biến kết quả
    result = {'N': [], 'V': []}
    
    # Tokenize câu
    word_list = ViTokenizer.tokenize(sentence).split()
    
    # Xử lý từng từ trong câu
    for word in word_list:
        try:
            word_str, pos_tag = toString(word, model, bi_grams, tri_grams, tagsetDict, inverse_tagsetDict)
            result[pos_tag].append(word_str)
        except Exception:
            pass  # Bỏ qua các từ không thể xử lý
            
    return result

## Running Pipeline

In [16]:
import pyaudio
import wave
import IPython.display as ipd
import os

In [27]:
def record_audio(filename, duration=5, channels=1, rate=16000, chunk=1024, format=pyaudio.paInt16):
    # Khởi tạo PyAudio object
    audio = pyaudio.PyAudio()

    # Thiết lập các tham số cho việc ghi âm
    stream = audio.open(format=format, channels=channels, rate=rate, input=True, frames_per_buffer=chunk)

    print("Recording...")

    # Khởi tạo list để lưu trữ các frame của âm thanh
    frames = []

    # Ghi âm trong một khoảng thời gian nhất định
    for _ in range(int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    print("Finished recording.")

    # Dừng stream và đóng PyAudio object
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Lưu các frames ghi âm vào file âm thanh
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(audio.get_sample_size(format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))

# Ghi âm một file audio có tên là "recorded_audio.wav" trong 5 giây
record_audio("recorded_audio.wav", duration=10)

ipd.Audio('recorded_audio.wav')


Recording...
Finished recording.


In [28]:
# Nhận diện người nói
filename = "recorded_audio.wav"
audio_file_path = os.path.abspath(filename)

audio_file_embedding = inference.get_embedding(audio_file_path, speaker_recognition_model)

tri_distance = inference.compute_distance(tri_base_embedding, audio_file_embedding)
dat_distance = inference.compute_distance(dat_base_embedding, audio_file_embedding)
tuan_distance = inference.compute_distance(tuan_base_embedding, audio_file_embedding)
phat_distance = inference.compute_distance(phat_base_embedding, audio_file_embedding)

data_distance = [tri_distance, dat_distance, tuan_distance, phat_distance]
users = ["Trí", "Đạt", "Tuấn", "Phát"]
prediction = users[data_distance.index(min(data_distance))]
print("Người nói được nhận diện là:", prediction)

# Nhận diện tiếng nói
content = speech2text(audio_file_path, speech_recognition_model)
print("Nội dung của đoạn âm thanh:", content)

# Nhận diện từ loại
pos_dictionary = sentence_to_pos(content, postagging_model)
print("Từ loại trong câu:", pos_dictionary)


Người nói được nhận diện là: Trí
Nội dung của đoạn âm thanh: và em xin chào thầy duy em là đức trí nhóm trưởng của nhóm bbm em rất vui khi được học với thầy thực sự quá nhiệt tình và hướng dẫn một em rất nhiều em xin cảm ơn thầy duy.
Từ loại trong câu: {'N': ['em', 'thầy', 'em', 'đức', 'trí', 'nhóm', 'trưởng', 'nhóm', 'em', 'khi', 'thầy', 'em', 'em', 'thầy'], 'V': ['xin', 'chào', 'là', 'học', 'hướng_dẫn', 'xin', 'cảm_ơn']}
